In [1]:
from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
import threading
from time import sleep
import pandas as pd
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
cursor = conn.cursor()
cursor.execute("SELECT distinct code from public.maincode where listed = 'us100';")
conn.commit()
code_list_1 = [i[0] for i in cursor.fetchall()]
code_list = code_list_1 + ['QQQ', "VOO"]
cursor.execute("SELECT distinct code from public.maincode where listed = 'us30';")
conn.commit()
code_list_2 = [i[0] for i in cursor.fetchall()]
code_list = code_list + code_list_2
code_list = list(set(code_list))
cursor.execute("SELECT distinct code from public.maincode where listed = 'us500';")
conn.commit()
code_list_3 = [i[0] for i in cursor.fetchall()]
code_l = [i for i in code_list_3 if i not in code_list]
code_list = code_list + code_l[0:200-len(code_list)]
len(code_list)

200

In [ ]:
from tqdm import tqdm
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.df = []
        self.minute = -1
        self.count = 0
        self.req_id_to_contract = {}

    def realtimeBar(self, reqId, time, open_, high, low, close, volume, wap, count):
        timestamp_utc = pd.to_datetime(time, unit='s', utc=True)
        tz = 'America/New_York'
        da = timestamp_utc.tz_convert(tz).strftime("%Y-%m-%d %H:%M:%S")
        # print(close, da, reqId)
        code = code_list[reqId-1]
        if da[-2:] == '00':
            self.df.append([da, code, open_, high, low, close, wap, volume])
            
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 1232)
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
reqID = 1
for symbol in tqdm(code_list):
    # if symbol in ['IBM', 'CAT', 'CSCO']: continue
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    app.reqRealTimeBars(reqId=reqID, contract=contract, barSize=5, whatToShow="TRADES", useRTH=0, realTimeBarsOptions=[])
    reqID += 1

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
  0%|          | 0/200 [00:00<?, ?it/s]ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj
100%|██████████| 200/200 [00:00<00:00, 11694.37it/s]


ERROR 5 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 143.14 to 143
ERROR 8 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 36.9 to 36
ERROR 14 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 297.47 to 297
ERROR 9 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 109.02 to 109
ERROR 12 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 2.94 to 2
ERROR 13 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 18.51 to 18
ERROR 16 2176 Warning: Your API version does not support fractional share size rul

In [3]:
import pandas as pd
import threading
import time
from IPython.display import clear_output
from datetime import datetime
import psycopg2
class ContinuousProcessor:
    def __init__(self):
        self.index_da = None
        self.lock = threading.Lock()
        self.running = True
        self.reqId = 250
        
        self.conn = self.get_db_connection()
        self.cursor = self.conn.cursor()

        self.reqId += 1
    def get_db_connection(self):
        DB_HOST = 'localhost'
        DB_NAME = 'us'
        DB_USER = 'postgres'
        DB_PASS = 'buddyrich134'
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        return conn
    def backtest_code(self, pivoted, code, rolling=5, benchmark_index='QQQ'):
        return None
    def orderStatus(self, orderId, status: str, filled,
                         remaining, avgFillPrice: float, permId: int,
                         parentId: int, lastFillPrice: float, clientId: int,
                         whyHeld: str, mktCapPrice: float):
        super().orderStatus(orderId, status, filled, remaining,
                            avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice)
        print(orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, whyHeld)
    def continuous_loop(self):
        while self.running:
            clear_output(wait=True)
            if len(app.df) == 0: 
                # print("market not open, app.df has len 0")
                # print(pd.to_datetime(time.time()*1000000000))
                time.sleep(10)
                continue
            df = pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'op', 'hi', 'lo', 'cl', 'wap', 'volume'])
            index = df['da'].max()
            if len(df) >= 200 and self.index_da != index:
                print(datetime.now())
                self.index_da = index
                time.sleep(3)
                df = pd.DataFrame(app.df[-2000:], columns=['da', 'code', 'op', 'hi', 'lo', 'cl', 'wap', 'volume']) 
                df = df.drop_duplicates(subset=['da', 'code'])
                insert_df = df[pd.to_datetime(df['da']) == index]
                self.cursor.executemany("INSERT INTO public.stock_price_5m (da, code, op, hi, lo, cl, wap, vol) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);", insert_df.values.tolist())
                self.conn.commit()
                print(datetime.now())
            else:
                time.sleep(10)
                ####

    def stop(self):
        self.running = False

processor = ContinuousProcessor()
loop_thread = threading.Thread(target=processor.continuous_loop, daemon=True)
loop_thread.start()

2024-10-17 21:36:06.151726
2024-10-17 21:36:09.187456
2024-10-17 21:37:09.227021
2024-10-17 21:37:12.259550
2024-10-17 21:38:12.305609
2024-10-17 21:38:15.339250
2024-10-17 21:39:15.382756
2024-10-17 21:39:18.412721
2024-10-17 21:40:08.454142
2024-10-17 21:40:11.483980
2024-10-17 21:41:11.523872
2024-10-17 21:41:14.560350
2024-10-17 21:42:14.585144
2024-10-17 21:42:17.613539
2024-10-17 21:43:07.655171
2024-10-17 21:43:10.692274
2024-10-17 21:44:10.736552
2024-10-17 21:44:13.784060
2024-10-17 21:45:13.833549
2024-10-17 21:45:16.883659
2024-10-17 21:46:06.919716
2024-10-17 21:46:09.951130
2024-10-17 21:47:10.001090
2024-10-17 21:47:13.045995
2024-10-17 21:48:13.095865
2024-10-17 21:48:16.133534
2024-10-17 21:49:06.178533
2024-10-17 21:49:09.217511
2024-10-17 21:50:09.272985
2024-10-17 21:50:12.313039
2024-10-17 21:51:12.366963
2024-10-17 21:51:15.407989
2024-10-17 21:52:05.455494
2024-10-17 21:52:08.503204
2024-10-17 21:53:08.560797
2024-10-17 21:53:11.603037
2024-10-17 21:54:11.666971
2

In [5]:
len(app.df), app.df[:1], app.df[-1:], len(list(set([i[1] for i in app.df])))

(200,
 [['2024-10-17 09:36:00', 'GOOGL', 165.31, 165.36, 165.3, 165.31, 165.33, 32]],
 [['2024-10-17 09:36:00', 'FAST', 77.73, 77.73, 77.73, 77.73, 77.73, 0]],
 200)